In [1]:
!pip install accelerate -U
!pip install transformers[torch]
! pip install datasets
! pip install --upgrade tqdm
!pip install pytorch-lightning

### data preparation

In [ ]:
# !pip install transformers

In [ ]:
# !pip install pillow

In [4]:
import json
data_path = "C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask2a\\train.json"
# Load your data
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

### data validation

In [7]:
new_data= []

In [10]:
import os
imgs_lst=  os.listdir("C:\\Users\\harih\\Downloads\\27\\train_images\\train_images")
#print(imgs_lst)

In [12]:
new_data=[dict_item for dict_item in data if dict_item["image"] in imgs_lst]

In [14]:
# Initialize an empty set to store unique labels
unique_labels = set()

# Iterate over each dictionary in the JSON data
for item in new_data:
    # Extract the "labels" field from the dictionary
    labels = item['labels']
    # Add the labels to the set
    unique_labels.update(labels)

# Convert the set of unique labels back to a list
unique_labels_list = list(unique_labels)

print("Unique labels in the data corpus:")
print(unique_labels_list)
print(len(unique_labels_list))

Unique labels in the data corpus:
['Exaggeration/Minimisation', 'Bandwagon', "Misrepresentation of Someone's Position (Straw Man)", 'Transfer', 'Appeal to (Strong) Emotions', 'Name calling/Labeling', 'Presenting Irrelevant Data (Red Herring)', 'Appeal to authority', 'Appeal to fear/prejudice', 'Doubt', 'Obfuscation, Intentional vagueness, Confusion', 'Whataboutism', 'Reductio ad hitlerum', 'Smears', 'Slogans', 'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification', 'Thought-terminating cliché', 'Repetition', 'Glittering generalities (Virtue)', 'Loaded Language', 'Flag-waving']
22


In [16]:
import pandas as pd
# Create a DataFrame
# Remove newline characters from the 'text' field
for item in new_data:
    item['text'] = item['text'].replace('\\n', ' ')

# Specify column names
column_names = ['id', 'text', 'image', 'labels']

# Convert JSON data to DataFrame
df = pd.DataFrame(new_data, columns=column_names)
pd.set_option('display.expand_frame_repr', False)

# Display the DataFrame
print(df.head())

print(f"num of records: {len(df)}")

      id                                               text                image                                             labels
0  63292    This is why we're free  This is why we're safe     prop_meme_556.png  [Causal Oversimplification, Transfer, Flag-wav...
1  65635  THIS IS WHY YOU NEED  A SHARPIE WITH YOU AT AL...   prop_meme_4839.png  [Transfer, Black-and-white Fallacy/Dictatorshi...
2  67927  GOOD NEWS!  NAZANIN ZAGHARI-RATCLIFFE AND ANOO...   prop_meme_7653.png  [Loaded Language, Glittering generalities (Vir...
3  68031                            PAING PHYO MIN IS FREE!   prop_meme_7826.png                 [Glittering generalities (Virtue)]
4  77490  Move your ships away!  oooook  Move your ships...  prop_meme_18807.png                                           [Smears]
num of records: 7000


In [18]:
# encode categories into labele columns
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(df.labels)
labels

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [20]:

len(labels)

7000

In [22]:
df = pd.concat([df[["id", 'text',"image"]], pd.DataFrame(labels)], axis=1)
df.columns = ["id",'text',"image"] + list(mlb.classes_)
df.head(4)

,id,text,image,Appeal to (Strong) Emotions,Appeal to authority,Appeal to fear/prejudice,Bandwagon,Black-and-white Fallacy/Dictatorship,Causal Oversimplification,Doubt,...,Name calling/Labeling,"Obfuscation, Intentional vagueness, Confusion",Presenting Irrelevant Data (Red Herring),Reductio ad hitlerum,Repetition,Slogans,Smears,Thought-terminating cliché,Transfer,Whataboutism
0,63292,This is why we're free This is why we're safe,prop_meme_556.png,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,65635,THIS IS WHY YOU NEED A SHARPIE WITH YOU AT AL...,prop_meme_4839.png,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
2,67927,GOOD NEWS! NAZANIN ZAGHARI-RATCLIFFE AND ANOO...,prop_meme_7653.png,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,68031,PAING PHYO MIN IS FREE!,prop_meme_7826.png,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:

from sklearn.model_selection import train_test_split
# split into train and test
df_train, df_test = train_test_split(df, random_state=77, test_size=0.30, shuffle=True)

In [26]:

print(f"Train: {df_train.shape}, Test: {df_test.shape}")


Train: (4900, 25), Test: (2100, 25)


In [28]:
target_list = list(df.columns)
target_list

['id',
 'text',
 'image',
 'Appeal to (Strong) Emotions',
 'Appeal to authority',
 'Appeal to fear/prejudice',
 'Bandwagon',
 'Black-and-white Fallacy/Dictatorship',
 'Causal Oversimplification',
 'Doubt',
 'Exaggeration/Minimisation',
 'Flag-waving',
 'Glittering generalities (Virtue)',
 'Loaded Language',
 "Misrepresentation of Someone's Position (Straw Man)",
 'Name calling/Labeling',
 'Obfuscation, Intentional vagueness, Confusion',
 'Presenting Irrelevant Data (Red Herring)',
 'Reductio ad hitlerum',
 'Repetition',
 'Slogans',
 'Smears',
 'Thought-terminating cliché',
 'Transfer',
 'Whataboutism']

In [30]:
target_list = target_list[3:]
print(target_list)

['Appeal to (Strong) Emotions', 'Appeal to authority', 'Appeal to fear/prejudice', 'Bandwagon', 'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification', 'Doubt', 'Exaggeration/Minimisation', 'Flag-waving', 'Glittering generalities (Virtue)', 'Loaded Language', "Misrepresentation of Someone's Position (Straw Man)", 'Name calling/Labeling', 'Obfuscation, Intentional vagueness, Confusion', 'Presenting Irrelevant Data (Red Herring)', 'Reductio ad hitlerum', 'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché', 'Transfer', 'Whataboutism']


In [32]:
targets= df[target_list].values
targets

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [34]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=True):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed.
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)

    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)
    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1

    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))

In [36]:
class_weights_dict= generate_class_weights(targets)

In [38]:
class_weights_dict

{0: 0.8576329331046312,
 1: 0.3632212536322125,
 2: 1.1570247933884297,
 3: 3.280224929709466,
 4: 0.533862111043319,
 5: 1.893939393939394,
 6: 1.1164274322169059,
 7: 1.3202565069784986,
 8: 0.7039420756234915,
 9: 0.8863003291972651,
 10: 0.43055726411612744,
 11: 10.9717868338558,
 12: 0.6300630063006301,
 13: 9.090909090909092,
 14: 21.21212121212121,
 15: 7.070707070707071,
 16: 6.769825918762089,
 17: 1.6486104569006124,
 18: 0.22502250225022502,
 19: 6.118881118881119,
 20: 1.9886363636363635,
 21: 6.91699604743083}

In [40]:
class_weights=   [value for key, value in class_weights_dict.items()]

In [42]:
class_weights

[0.8576329331046312,
 0.3632212536322125,
 1.1570247933884297,
 3.280224929709466,
 0.533862111043319,
 1.893939393939394,
 1.1164274322169059,
 1.3202565069784986,
 0.7039420756234915,
 0.8863003291972651,
 0.43055726411612744,
 10.9717868338558,
 0.6300630063006301,
 9.090909090909092,
 21.21212121212121,
 7.070707070707071,
 6.769825918762089,
 1.6486104569006124,
 0.22502250225022502,
 6.118881118881119,
 1.9886363636363635,
 6.91699604743083]

### Tokenization

In [44]:
from transformers import BertTokenizer, VisualBertForPreTraining, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

C:\Users\harih\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Test Tokenization

### visual feature extractor

In [46]:

# Test the tokenizer
test_text = "We are testing BERT tokenizer."
# generate encodings
encodings = tokenizer.encode_plus(test_text,
                                  add_special_tokens = True,
                                  max_length = 128,
                                  truncation = True,
                                  padding = "max_length",
                                  return_attention_mask = True,
                                  return_tensors = "pt")
# we get a dictionary with three keys (see: https://huggingface.co/transformers/glossary.html)
encoded_dict = encodings

In [48]:
import torch
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
import numpy as np

In [58]:
# Load model directly
from transformers import AutoImageProcessor, AutoModel

processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
model = AutoModel.from_pretrained("google/vit-base-patch16-224-in21k")

In [61]:
!pip install transformers



In [64]:

import requests




In [68]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
feature_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k').to('cpu')

C:\Users\harih\.conda\envs\NLP\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [71]:
from transformers import BertTokenizer, VisualBertModel, TrainingArguments, Trainer, VisualBertConfig
configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre',
                                                hidden_dropout_prob=0.3, attention_probs_dropout_prob=0.3)
#model = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre', config=configuration)

In [75]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

class HatefulMemesData(Dataset):
    def __init__(self, df, tokenizer, sequence_length, target_list, print_text=False):
        self.sequence_length = sequence_length
        self.tokenizer = tokenizer
        self.print_text = print_text

        texts = df["text"].values.tolist()
        labels = df[target_list].values.tolist()  # Change to "labels" column for multi-label
        images = df["image"].values.tolist()
        ids =  df["id"].values.tolist()  # Change to "id" column

        self.dataset = []
        for i, inp in enumerate(texts):
            self.dataset.append({"text": inp, "labels": labels[i], 'id': ids[i], 'image': images[i]})

    def __len__(self):
        return len(self.dataset)

    def tokenize_data(self, example):
        idx = example['id']
        idx = [idx] if isinstance(idx, str) else idx

        encoded_dict = self.tokenizer(example['text'], padding='max_length', max_length=self.sequence_length, truncation=True, return_tensors='pt')
        tokens = encoded_dict['input_ids']
        token_type_ids = encoded_dict['token_type_ids']
        attn_mask = encoded_dict['attention_mask']

        targets = torch.tensor(example['labels'], dtype=torch.int64)  # Change to float for multi-label

        # Get Visual Embeddings
        try:
            img = example['image']
            img_path = os.path.join('C:\\Users\\harih\\Downloads\\27\\train_images\\train_images', img)
            img = Image.open(img_path)

            if img.mode != 'RGB':
                img = img.convert('RGB')

            img = np.array(img)
            inputs = feature_extractor(images=img, return_tensors="pt")
            outputs = feature_model(**inputs.to('cpu'))
            visual_embeds = outputs.last_hidden_state
            visual_embeds = visual_embeds.cpu()
        except Exception as e:
            print("Error with Id: ", idx, e)
            visual_embeds = torch.zeros(197, 768)  # Adjust the shape according to your model

        visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)

        inputs = {
            "input_ids": tokens.squeeze(),
            "attention_mask": attn_mask.squeeze(),
            "token_type_ids": token_type_ids.squeeze(),
            "visual_embeds": visual_embeds.squeeze(),
            "visual_token_type_ids": visual_token_type_ids.squeeze(),
            "visual_attention_mask": visual_attention_mask.squeeze(),
            "labels": targets.squeeze()
        }

        return inputs

    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, inputs[k].shape, inputs[k].dtype)

        return inputs

In [77]:
dataset = HatefulMemesData(df, tokenizer, 250, target_list, True)


In [79]:
len(dataset)

7000

In [81]:
df["id"][1]

'65635'

In [ ]:
# !pip install transformers --upgrade

In [83]:
dataset[10]

input_ids torch.Size([250]) torch.int64
attention_mask torch.Size([250]) torch.int64
token_type_ids torch.Size([250]) torch.int64
visual_embeds torch.Size([197, 768]) torch.float32
visual_token_type_ids torch.Size([197]) torch.int64
visual_attention_mask torch.Size([197]) torch.int64
labels torch.Size([22]) torch.int64


{'input_ids': tensor([ 101, 3407, 2047, 2095, 2013, 2343, 6221, 1046, 1012, 8398,  102,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

In [85]:
example1 = dataset[10]

input_ids torch.Size([250]) torch.int64
attention_mask torch.Size([250]) torch.int64
token_type_ids torch.Size([250]) torch.int64
visual_embeds torch.Size([197, 768]) torch.float32
visual_token_type_ids torch.Size([197]) torch.int64
visual_attention_mask torch.Size([197]) torch.int64
labels torch.Size([22]) torch.int64


### Training

In [88]:
from transformers import BertTokenizer, VisualBertModel, TrainingArguments, Trainer, VisualBertConfig
configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre',
                                                hidden_dropout_prob=0.3, attention_probs_dropout_prob=0.3)
model = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre', config=configuration)

In [90]:
from transformers import TrainingArguments, Trainer
batch_size = 2
seq_len = 250

In [92]:
print(model)

VisualBertModel(
  (embeddings): VisualBertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (visual_token_type_embeddings): Embedding(2, 768)
    (visual_position_embeddings): Embedding(512, 768)
    (visual_projection): Linear(in_features=1024, out_features=768, bias=True)
  )
  (encoder): VisualBertEncoder(
    (layer): ModuleList(
      (0-11): 12 x VisualBertLayer(
        (attention): VisualBertAttention(
          (self): VisualBertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.3, inplace=False)
          )
          (output): V

In [94]:
model = model.double()

In [ ]:
# ! pip install transformers --upgrade
# ! pip install datasets
# ! pip install --upgrade tqdm

In [96]:
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from datasets import load_metric
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from transformers import (
    AdamW,
    VisualBertModel,
    get_linear_schedule_with_warmup
)
import logging
import argparse
import time
from torch.nn import CrossEntropyLoss
from sklearn.metrics import roc_auc_score


In [98]:
# from pytorch_lightning.loggers.wandb import WandbLogger
import os
from pathlib import Path
from string import punctuation
import torch.nn as nn

In [100]:
import torch
from transformers import VisualBertModel, VisualBertConfig

In [102]:
import random

dataset_size = 1250
num_classes = 22

# Randomly select 22 indices from the dataset
random_indices = random.sample(range(dataset_size), num_classes)

print(random_indices)

[1137, 355, 988, 134, 194, 1145, 237, 1002, 369, 517, 976, 901, 325, 1167, 460, 825, 296, 439, 251, 1054, 304, 990]


In [ ]:
# import torch.nn.functional as F

# class VisualBERTClassifier(torch.nn.Module):
#     def __init__(self, class_weights):
#         super(VisualBERTClassifier, self).__init__()
#         configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre',
#                                                 hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1)
#         self.visualbert = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre', config=configuration)
#         self.embed_cls = nn.Linear(768, 1024)
#         self.num_labels = 22
#         self.dropout = nn.Dropout(0.3)
#         self.cls = nn.Linear(768, self.num_labels)

#     def forward(self, input_ids, attention_mask, token_type_ids, visual_embeds, visual_attention_mask,
#                 visual_token_type_ids, labels):
#         visual_embeds_cls = self.embed_cls(visual_embeds)
#         outputs = self.visualbert(
#                 input_ids,
#                 attention_mask=attention_mask,
#                 token_type_ids=token_type_ids,
#                 visual_embeds=visual_embeds_cls,
#                 visual_attention_mask=visual_attention_mask,
#                 visual_token_type_ids=visual_token_type_ids,
#             )

#         pooled_output = outputs[1]
#         pooled_output = self.dropout(pooled_output)
#         logits = self.cls(pooled_output)

#         # Apply sigmoid activation
#         probabilities = torch.sigmoid(logits)

#         # Compute binary cross-entropy loss
#         loss_fct = nn.BCEWithLogitsLoss(weight=torch.FloatTensor(class_weights))
#         loss = loss_fct(probabilities, labels.float())

#         return loss, logits


In [104]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import VisualBertConfig, VisualBertModel

# Set the device to GPU if available; otherwise, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class VisualBERTClassifier(nn.Module):
    def __init__(self, class_weights):
        super(VisualBERTClassifier, self).__init__()

        # Configuration with dropout settings
        configuration = VisualBertConfig.from_pretrained(
            'uclanlp/visualbert-nlvr2-coco-pre',
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1
        )

        # Initialize the VisualBERT model and additional layers
        self.visualbert = VisualBertModel.from_pretrained(
            'uclanlp/visualbert-nlvr2-coco-pre', config=configuration
        )
        self.embed_cls = nn.Linear(768, 1024)  # Embedding linear layer for visual data
        self.dropout = nn.Dropout(0.3)  # Dropout layer
        self.cls = nn.Linear(768, 22)  # Final classification layer with 22 output classes

        # Ensure class weights are on the correct device
        self.class_weights = torch.FloatTensor(class_weights).to(device)

    def forward(
        self,
        input_ids,
        attention_mask,
        token_type_ids,
        visual_embeds,
        visual_attention_mask,
        visual_token_type_ids,
        labels
    ):
        # Move inputs to the correct device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        visual_embeds = visual_embeds.to(device)
        visual_attention_mask = visual_attention_mask.to(device)
        visual_token_type_ids = visual_token_type_ids.to(device)
        labels = labels.to(device)

        # Additional processing for visual embeddings
        visual_embeds_cls = self.embed_cls(visual_embeds)

        # Forward pass with VisualBERT
        outputs = self.visualbert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            visual_embeds=visual_embeds_cls,
            visual_attention_mask=visual_attention_mask,
            visual_token_type_ids=visual_token_type_ids,
        )

        # Dropout and final linear layer
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.cls(pooled_output)

        # Binary cross-entropy loss
        loss_fct = nn.BCEWithLogitsLoss(weight=self.class_weights)
        loss = loss_fct(logits, labels.float())

        return loss, logits

In [106]:
model = VisualBERTClassifier(class_weights=random_indices).to("cpu")

In [120]:
metric_name = "auroc"

args = TrainingArguments(
    output_dir = "model-checkpoint",
    seed = 110,
    evaluation_strategy = "steps",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 10,
    weight_decay=0.05,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    eval_steps = 250,
    save_steps = 500,
    fp16 = False,
    gradient_accumulation_steps = 2


)

In [110]:
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
print(metrics_list)

['accuracy', 'bertscore', 'bleu', 'bleurt', 'brier_score', 'cer', 'character', 'charcut_mt', 'chrf', 'code_eval', 'comet', 'competition_math', 'confusion_matrix', 'coval', 'cuad', 'exact_match', 'f1', 'frugalscore', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis', 'mape', 'mase', 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'nist_mt', 'pearsonr', 'perplexity', 'poseval', 'precision', 'r_squared', 'recall', 'rl_reliability', 'roc_auc', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'smape', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'ter', 'trec_eval', 'wer', 'wiki_split', 'xnli', 'xtreme_s', 'Aledade/extraction_evaluation', 'AlhitawiMohammed22/CER_Hu-Evaluation-Metrics', 'Bekhouche/NED', 'BucketHeadP65/confusion_matrix', 'BucketHeadP65/roc_curve', 'CZLC/rouge_raw', 'DaliaCaRo/accents_unplugged_eval', 'DarrenChensformer/action_generation', 'DarrenChensformer/eval_keyphrase', 'DarrenChensformer/relation_extraction', 'DoctorSlimm/bangalore_score', 'DoctorSlimm/ka

C:\Users\harih\AppData\Local\Temp\ipykernel_16556\2537104428.py:2: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


In [112]:
acc_metric = load_metric('accuracy')
f1_metric = load_metric('f1')
precision_metric = load_metric('precision')
recall_metric = load_metric('recall')

C:\Users\harih\AppData\Local\Temp\ipykernel_16556\2965749901.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  acc_metric = load_metric('accuracy')
C:\Users\harih\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
C:\Users\harih\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:759: FutureWarning: The repository for f1 contains custom code w

In [114]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

def compute_multilabel_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = (logits > 0.5).astype(int)  # Assuming a threshold of 0.5 for binary prediction

    acc = accuracy_score(labels, predictions)
    auc_score = roc_auc_score(labels, logits)  # Use logits for AUC calculation
    f1 = f1_score(labels, predictions, average='micro')  # Compute F1 score with micro averaging
    precision = precision_score(labels, predictions, average='micro')  # Compute precision with micro averaging
    recall = recall_score(labels, predictions, average='micro')  # Compute recall with micro averaging

    return {"accuracy": acc, "auroc": auc_score, 'f1': f1, 'precision': precision, 'recall': recall}


In [124]:
trainer = Trainer(
    model,
    args,
    train_dataset = HatefulMemesData(df_train,tokenizer=tokenizer, sequence_length=seq_len,target_list= target_list),
    eval_dataset =  HatefulMemesData(df_test,tokenizer=tokenizer, sequence_length=seq_len, target_list= target_list),
    tokenizer=tokenizer,
    compute_metrics=compute_multilabel_metrics
)

In [126]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Auroc,F1,Precision,Recall
250,No log,154.212982,0.000000,0.559737,0.000000,0.000000,0.000000
500,155.819300,152.927765,0.000000,0.564009,0.000000,0.000000,0.000000
750,155.819300,152.228302,0.000000,0.586766,0.000000,0.000000,0.000000
1000,148.781000,149.991760,0.000000,0.595896,0.000000,0.000000,0.000000
1250,148.781000,150.094345,0.000000,0.578113,0.000000,0.000000,0.000000
1500,149.162200,147.828995,0.000000,0.602925,0.004163,0.769231,0.002087
1750,149.162200,144.970749,0.000000,0.613935,0.009123,0.687500,0.004592
2000,145.337700,144.722580,0.000952,0.615912,0.057336,0.620690,0.030056
2250,145.337700,144.182144,0.002381,0.616536,0.070713,0.600000,0.037570
2500,140.159800,143.633072,0.003810,0.626473,0.099981,0.559574,0.054895


C:\Users\harih\.conda\envs\NLP\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\harih\.conda\envs\NLP\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\harih\.conda\envs\NLP\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\harih\.conda\envs\NLP\Lib\site-packag

TrainOutput(global_step=12250, training_loss=113.67605500637755, metrics={'train_runtime': 25111.6956, 'train_samples_per_second': 1.951, 'train_steps_per_second': 0.488, 'total_flos': 0.0, 'train_loss': 113.67605500637755, 'epoch': 10.0})

In [131]:
eval_dataset =  HatefulMemesData(df_test,tokenizer=tokenizer, sequence_length=seq_len, target_list= target_list)

In [133]:
eval_dataset[0]

{'input_ids': tensor([ 101, 1996, 2755, 2008, 2017, 2655, 2009, 1032, 9714, 3399, 1032, 7127,
         2008, 2017, 2031, 2070, 3752, 2000, 2079,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

###  test for multi label classification

In [135]:
class VisualBERTClassifier(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(VisualBERTClassifier, self).__init__()
        configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre',
                                                hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1)
        self.visualbert = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre', config=configuration)
        self.embed_cls = nn.Linear(768, 1024)
        # self.visualbert = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre')
        self.num_labels = 2
        self.dropout = nn.Dropout(0.3)
        self.cls=  nn.Linear(768, self.num_labels)
        self.weight = torch.FloatTensor([class_weights]) #torch.FloatTensor([0.77510622, 1.40873991]),
        nSamples = random_indices
        normedWeights = [1 - (x / sum(nSamples)) for x in nSamples]
        self.loss_fct = CrossEntropyLoss(weight=torch.FloatTensor(normedWeights))


    def forward(self, input_ids, attention_mask, token_type_ids, visual_embeds, visual_attention_mask,
                visual_token_type_ids, labels):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        visual_embeds_cls = self.embed_cls(visual_embeds)
        outputs = self.visualbert(
                input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                visual_embeds=visual_embeds_cls,
                visual_attention_mask=visual_attention_mask,
                visual_token_type_ids=visual_token_type_ids,
            )

        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.cls(pooled_output)
        reshaped_logits = logits.view(-1, self.num_labels)

        loss = self.loss_fct(reshaped_logits, labels.view(-1))

        return loss, logits

In [137]:
example= dataset[0]

input_ids torch.Size([250]) torch.int64
attention_mask torch.Size([250]) torch.int64
token_type_ids torch.Size([250]) torch.int64
visual_embeds torch.Size([197, 768]) torch.float32
visual_token_type_ids torch.Size([197]) torch.int64
visual_attention_mask torch.Size([197]) torch.int64
labels torch.Size([22]) torch.int64


In [139]:
example

{'input_ids': tensor([ 101, 2023, 2003, 2339, 2057, 1005, 2128, 2489, 2023, 2003, 2339, 2057,
         1005, 2128, 3647,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

In [147]:
model = VisualBERTClassifier().to('cpu')